# Setup

In [10]:
%profile "cloudbend"

%glue_version "3.0"
%iam_role "arn:aws:iam::898546127587:role/GlueSessions"
%idle_timeout 60

%worker_type "G.1X"
%number_of_workers 5

%extra_jars "s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar"

Previous profile: default
Setting new profile to: cloudbend
Setting Glue version to: 3.0
Current iam_role is None
iam_role has been set to arn:aws:iam::898546127587:role/GlueSessions.
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Extra jars to be included:
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-cl

In [12]:
%%configure
{
    "conf": "spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
}

You are already connected to a glueetl session 14f528a4-1625-4e1e-8b20-233004ec35ab.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


The following configurations have been updated: {'conf': 'spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'}


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

In [3]:
catalog = "iceberg"
bucket = "lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob"
prefix = "data"

iceberg_conf = [
    (f"spark.sql.catalog.{catalog}", "org.apache.iceberg.spark.SparkCatalog"),
    (f"spark.sql.catalog.{catalog}.warehouse", f"s3://{bucket}/{prefix}/{catalog}/"),
    (f"spark.sql.catalog.{catalog}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog"),
    (f"spark.sql.catalog.{catalog}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"),
]

for k, v in iceberg_conf:
    spark.conf.set(k, v)

#

In [4]:
df = spark.table("er.nc_voter")

df.count()

3961042


In [5]:
df.createOrReplaceTempView("raw")

In [6]:
%%sql

CREATE DATABASE IF NOT EXISTS iceberg.demo;

++
||
++
++


In [7]:
%%sql

CREATE TABLE iceberg.demo.nc_voter
USING iceberg
TBLPROPERTIES (
    'write.parquet.compression-codec' = 'snappy',
    'write.metadata.metrics.default' = 'full'
)
AS
SELECT *
FROM raw
ORDER BY LOWER(name) ASC;

++
||
++
++


In [8]:
%%sql

SELECT *
FROM iceberg.demo.nc_voter
LIMIT 10;

+------------------+--------------------+
|              name|               votes|
+------------------+--------------------+
|      'lmam Eliham|[{7862734, lelamd...|
|   'lngram Antonia|[{8064965, wake f...|
|   'lngrao Kri5ten|[{5549424, greens...|
|'lnman Dannalaural|[{4717549, la gra...|
|      'lnscoe Joan|[{7218106, lelamd...|
| .magruder William|[{2759006, blowin...|
| .wilkins Corriell|[{6884779, rocky ...|
|      0'hara Shaun|[{5561290, edento...|
|     0'neal Alicia|[{4197887, chapel...|
|         0ace John|[{2943144, peach1...|
+------------------+--------------------+


In [14]:
%stop_session

Stopping session: 14f528a4-1625-4e1e-8b20-233004ec35ab
Stopped session.
